### CNN 학습 데이터 만들기
### Yolov5로 crop -> 흑백 -> 좌우flip 증강 -> 저장

In [2]:
import os
import cv2
import torch
from tqdm import tqdm

for i in range(5):
    os.makedirs('./train_cnn/1. yolo_detect/{}'.format(i), exist_ok = True)
model = torch.hub.load('ultralytics/yolov5', 'custom', path = './yolov5-master/exp3/weights/best.pt')
video_folder_path = './train_video'
folder_list = os.listdir(video_folder_path)
for folder_name in folder_list:
    video_list = os.listdir('{}/{}'.format(video_folder_path, folder_name))
    name_cnt = 0
    for video_name in tqdm(video_list):
        cap = cv2.VideoCapture('{}/{}/{}'.format(video_folder_path, folder_name, video_name))
        if cap.isOpened():
            img_list = []
            while True:
                ret, img = cap.read()
                if ret == True:
                    img_list.append(img)
                else: break
            img_list = img_list[int(len(img_list) / 3 * 2):]
            for img in img_list:
                pred_list = model(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                pred_list = pred_list.pandas().xyxy[0]
                result_list = []
                for cnt in range(len(pred_list)):
                    x1 = int(pred_list.loc[cnt]['xmin'])
                    y1 = int(pred_list.loc[cnt]['ymin'])
                    x2 = int(pred_list.loc[cnt]['xmax'])
                    y2 = int(pred_list.loc[cnt]['ymax'])
                    conf = round(float(pred_list.loc[cnt]['confidence']), 3)
                    cls_no = pred_list.loc[cnt]['class']
                    cls_name = pred_list.loc[cnt]['name']
                    pred = {'bbox' : [x1, y1, x2, y2], 'conf' : conf, 'cls_no' : cls_no, 'cls_name' : cls_name}
                    result_list.append(pred)
                if len(result_list) > 0:
                    result_list.sort(key = lambda x:x['conf'], reverse = True)
                    result = result_list[0]
                    crop_img = img[result['bbox'][1]:result['bbox'][3], result['bbox'][0]:result['bbox'][2]]
#                     crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                    cv2.imwrite('./train_cnn/1. yolo_detect/{}/{}.png'.format(folder_name, name_cnt), crop_img)
                    flip_crop_img = cv2.flip(crop_img, 1)
                    cv2.imwrite('./train_cnn/1. yolo_detect/{}/{}_flip.png'.format(folder_name, name_cnt), flip_crop_img)
                    name_cnt += 1
                else: continue

Using cache found in C:\Users\nyan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-15 Python-3.8.15 torch-1.10.1 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients
Adding AutoShape... 
100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [01:00<00:00,  1.89it/s]


### 2번 3번 class 합쳐서 데이터 밸런스 지키기

In [3]:
# 0, 1, 2, 3, 4 class가 0, 1, 2(2+3), 3으로 변함
# 1. yolo_detect 폴더를 2. 2 and 3 merge 폴더에 통채로 복사해 놓고 시작할 것
import os
import shutil
import cv2
import random
from tqdm import tqdm

folder_2 = './train_cnn/2. 2 and 3 merge/2'
folder_3 = './train_cnn/2. 2 and 3 merge/3'
os.makedirs('./train_cnn/2. 2 and 3 merge/new_2and3', exist_ok = True)
img_list = []
img_name_list = os.listdir(folder_2)
for img_name in tqdm(img_name_list):
    img = cv2.imread('{}/{}'.format(folder_2, img_name))
    img_list.append(img)
img_name_list = os.listdir(folder_3)
for img_name in tqdm(img_name_list):
    img = cv2.imread('{}/{}'.format(folder_3, img_name))
    img_list.append(img)
    
random.shuffle(img_list)
print('2와 3을 더한 img list 길이: {}'.format(len(img_list)))
img_list = img_list[:int(len(img_list)/2)]
print('절반으로 나눈 길이: {}'.format(len(img_list)))
name_cnt = 0
for img in tqdm(img_list):
    cv2.imwrite('./train_cnn/2. 2 and 3 merge/new_2and3/{}.png'.format(name_cnt), img)
    name_cnt += 1
    
# 이후에 폴더 이름은 알아서 바꿔주기: 0, 1, 2, 3으로

100%|█████████████████████████████████████████████████████████████████████████████| 2558/2558 [00:11<00:00, 232.14it/s]


2와 3을 더한 img list 길이: 4772
절반으로 나눈 길이: 2386


100%|█████████████████████████████████████████████████████████████████████████████| 2386/2386 [00:02<00:00, 962.04it/s]


### classifier 학습

In [4]:
import torch

def seed(seed = 1234):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
if torch.cuda.is_available() == True:
    device = 'cuda:0'
    print('현재 가상환경 GPU 사용 가능상태')
else:
    device = 'cpu'
    print('GPU 사용 불가능 상태')

현재 가상환경 GPU 사용 가능상태


In [5]:
import splitfolders
input_folder = './train_cnn/2. 2 and 3 merge'
data_dir = './train_cnn/3. split_data'
splitfolders.ratio(input_folder, data_dir, seed=1, ratio=(0.9, 0.1, 0.0))

Copying files: 9484 files [00:46, 203.11 files/s]


In [6]:
seed()
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Resize, Normalize
import numpy as np
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = transforms.Compose([ToTensor(), Resize((224,224)), Normalize(mean, std)])

In [7]:
seed()
from torchvision import datasets
train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms)
val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms)

dataloader_train = torch.utils.data.DataLoader(train_data, batch_size = 16, shuffle = True)
dataloader_val = torch.utils.data.DataLoader(val_data, batch_size = 16)

In [8]:
def run_resnet(net, data_loader, loss_fn, optim, scheduler, mode = 'train'):
    global epoch_cnt
    iter_loss, iter_acc, last_grad_performed = [], [], False

    for _data, _label in data_loader:
        data, label = _data.to(device), _label.to(device)
        
        # 1. Feed-forward
        if mode == 'train':
            net.train()
            grad_mode = True
        else:
            # 학습때만 쓰이는 Dropout, Batch Mormalization을 미사용
            net.eval()
            grad_mode = False
        
        with torch.set_grad_enabled(grad_mode):
            result = net(data)
             
            # Feed Forward
            _, out = torch.max(result, 1) # result에서 최대 확률값을 기준으로 예측 class 도출
        
            # 2. Loss 계산
            loss = loss_fn(result, label) # GT 와 Label 비교하여 Loss 산정
            iter_loss.append(loss.item()) # 학습 추이를 위하여 Loss를 기록

            # 3. 역전파 학습 후 Gradient Descent
            if mode == 'train':
                optim.zero_grad() # 미분을 통해 얻은 기울기르 초기화 for 다음 epoch
                loss.backward() # 역전파 학습
                optim.step() # Gradient Descent 수행
                last_grad_performed = True # for문 나가면 epoch 카운터 += 1

            # 4. 정확도 계산
            acc_partial = (out == label).float().sum() # GT == Label 인 개수
            acc_partial = acc_partial / len(label) # ( TP / (TP + TN)) 해서 정확도 산출
            iter_acc.append(acc_partial.item()) # 학습 추이를 위하여 Acc. 기록
    
    ### 이번에 새로 배우는 scheduler
    scheduler.step() # Learning Rate 스케줄러 실행

    # 역전파 학습 후 Epoch 카운터 += 1
    if last_grad_performed:
        epoch_cnt += 1
    
    clear_memory()
    
    # loss와 acc의 평균값 for 학습추이 그래프
    return np.average(iter_loss), np.average(iter_acc)

In [9]:
seed()
from torchvision import models
import torch.nn as nn
from torchsummary import summary as Summary

model_resnet = models.resnet152(pretrained = True)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 4)
# model_resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
net = model_resnet.to(device)
Summary(model_resnet.to(device), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [10]:
import matplotlib.pyplot as plt
    
# epoch 카운터 초기화
def init_epoch():
    global epoch_cnt
    epoch_cnt = 0

def init_log():
    plt.rc('font', size = 10)
    # 모든 Log를 초기화
    global log_stack, iter_log, tloss_log, tacc_log, vloss_log, vacc_log, time_log
    iter_log, tloss_log, tacc_log, vloss_log, vacc_log = [], [], [], [], []
    time_log, log_stack = [], []

In [11]:
import gc
from torch.cuda import memory_allocated, empty_cache
def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()

def epoch_not_finished():
    # 에폭이 끝남을 알림
    return epoch_cnt < maximum_epoch

In [12]:
def record_train_log(_tloss, _tacc, _time):
    # Train Log 기록용
    time_log.append(_time)
    tloss_log.append(_tloss)
    tacc_log.append(_tacc)
    iter_log.append(epoch_cnt)
    
def record_valid_log(_vloss, _vacc):
    # Validation Log 기록용
    vloss_log.append(_vloss)
    vacc_log.append(_vacc)

def last(log_list):
    # 리스트 안의 마지막 숫자를 반환(print_log 함수에서 사용)
    if len(log_list) > 0:
        return log_list[len(log_list) - 1]
    else:
        return -1

from IPython.display import clear_output
def print_log():
    # 학습 추이 출력

    # 소숫점 3자리 수까지 조절
    train_loss = round(float(last(tloss_log)), 3)
    train_acc = round(float(last(tacc_log)), 3)
    val_loss = round(float(last(vloss_log)), 3)
    val_acc = round(float(last(vacc_log)), 3)
    time_spent = round(float(last(time_log)), 3)
    
    log_str = 'Epoch: {:3} | T_Loss {:5} | T_acc {:5} | V_Loss {:5} | V_acc. {:5} | \
🕒 {:5}'.format(last(iter_log), train_loss, train_acc, val_loss, val_acc, time_spent)
    
    log_stack.append(log_str) # 프린트 준비
    
    # 학습 추이 그래프 출력
    hist_fig, loss_axis = plt.subplots(figsize=(10, 3), dpi=99) # 그래프 사이즈 설정
    hist_fig.patch.set_facecolor('white') # 그래프 배경색 설정
    
    # Loss Line 구성
    loss_t_line = plt.plot(iter_log, tloss_log, label='Train Loss', color='red', marker='o')
    loss_v_line = plt.plot(iter_log, vloss_log, label='Valid Loss', color='blue', marker='s')
    loss_axis.set_xlabel('epoch')
    loss_axis.set_ylabel('loss')
    
    # Acc. Line 구성
    acc_axis = loss_axis.twinx()
    acc_t_line = acc_axis.plot(iter_log, tacc_log, label='Train Acc.', color='red', marker='+')
    acc_v_line = acc_axis.plot(iter_log, vacc_log, label='Valid Acc.', color='blue', marker='x')
    acc_axis.set_ylabel('accuracy')
    
    # 그래프 출력
    hist_lines = loss_t_line + loss_v_line + acc_t_line + acc_v_line # 위에서 선언한 plt정보들 통합
    loss_axis.legend(hist_lines, [l.get_label() for l in hist_lines], loc = 'upper right') # 순서대로 그려주기
    loss_axis.grid() # 격자 설정
    plt.title('Learning history until epoch {}'.format(last(iter_log)))
    plt.draw()
    
    # 텍스트 로그 출력
    clear_output(wait=True)
    plt.show()
    for idx in reversed(range(len(log_stack))): # 반대로 sort 시켜서 출력
        print(log_stack[idx])

In [13]:
seed()
import time
from torch.optim import lr_scheduler
from torch.optim import Adam

maximum_epoch = 3
loss_fn = nn.CrossEntropyLoss()
optim = Adam(net.parameters(), lr=0.0001)
lr_scheduler = lr_scheduler.StepLR(optim, step_size = 1, gamma = 1)
init_epoch()
init_log()
# Training Iteration
best_acc, best_loss = 0, 0
while epoch_not_finished():
    start_time = time.time()
    tloss, tacc = run_resnet(net, dataloader_train, loss_fn, optim, lr_scheduler, mode = 'train')
    end_time = time.time()
    time_taken = end_time - start_time
    record_train_log(tloss, tacc, time_taken)
    with torch.no_grad():
        vloss, vacc = run_resnet(net, dataloader_val, loss_fn, optim, lr_scheduler, mode = 'val')
        record_valid_log(vloss, vacc)
    print_log()
    if vacc > best_acc or vloss < best_loss:
        if vacc > best_acc: best_acc = vacc
        if vloss < best_loss: best_loss = vloss
        torch.save(net.state_dict(), './best_model_cnn.pth')
        print('최고점수 갱신 후 모델 저장: {}epoch'.format(epoch_cnt))
print('\n Training completed!')

Epoch:   3 | T_Loss 0.017 | T_acc 0.994 | V_Loss 0.021 | V_acc. 0.992 | 🕒 142.147
Epoch:   2 | T_Loss 0.031 | T_acc 0.991 | V_Loss  0.01 | V_acc. 0.996 | 🕒 142.31
Epoch:   1 | T_Loss 0.084 | T_acc 0.973 | V_Loss 0.006 | V_acc. 0.999 | 🕒 178.524

 Training completed!
